In [1]:
# import
import pandas as pd
import numpy as np
from numba import njit

from simple.chart import interactFigure, updateFigure, updateSliders, chartParallel
from simple.backtest import getProfitDict, npTrades
from simple.funcs import vwap, vPIN
from simple.geneopt import GeneOpt
from simple.types import TTrade, TPairTrade

from ipyslickgrid import show_grid
from multiprocessing import current_process
from ipywidgets import VBox

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
from numba.types import int64, float64, Tuple
from numpy.typing import NDArray
from numba.typed import List

fp32 = np.float32
default_fee = 0.05

signal_type = Tuple((int64, int64, float64, float64))
trade_type = Tuple((int64, int64, float64, float64, int64, int64, float64, float64, int64))

@njit(nogil=True)
def backtestLimit(T: NDArray[TTrade], qA: NDArray[float], qB: NDArray[float], signal, threshold, fee_percent=default_fee) -> List[trade_type]:
    """Vectorized backtester for limit order strategies with filter"""

    buys = List.empty_list(signal_type)
    sells = List.empty_list(signal_type)
    trades = List.empty_list(trade_type)
    ts = T.DateTime.view(np.int64)
    pos: int = 0

    for i in range(len(ts) - 1):
        price = T.Price[i]

        if price > qA[i]:
            delta_pos = -min(pos + 1, 1)
        elif price < qB[i]:
            delta_pos = min(1 - pos, 1)
        else:
            delta_pos = 0

        k = i + 1
        if delta_pos > 0:
            buys.append((k, ts[k], qB[i], qB[i]))
        elif delta_pos < 0:
            sells.append((k, ts[k], qA[i], qA[i]))

        if len(sells) > 0 and len(buys) > 0:
            buy = buys.pop(0)
            sell = sells.pop(0)
            if delta_pos < 0:
                trades.append((*buy, *sell, -delta_pos))
            else:
                trades.append((*sell, *buy, -delta_pos))

        pos += delta_pos

    return trades

In [4]:
# declare chart linestyles
line_styles = {
    'Center': dict(color='blue', opacity=0.5),
    'qA': dict(color='red', opacity=0.5, dash='dot'),
    'qB': dict(color='green', opacity=0.5, dash='dot'),
    'OSC': dict(color='orange', row=2, col=1, opacity=0.6)
}

In [5]:
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1), Threshold: int = (0, 40)):
    if Period == 0 or StdDev == 0: return {}
    Tick = T.Price
    Center = vwap(T, Period)
    _std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + _std*StdDev
    qB = Center - _std*StdDev
    OSC = np.nan_to_num(vPIN(T, Period))

    _trades = npTrades(backtestLimit(T, qA, qB, OSC, Threshold))
    return getProfitDict(_trades) if current_process().daemon else locals()

In [6]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=256, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 27222, 'StdDev': 2.3943980486597325, 'Threshold': 39}

In [7]:
# Create interactive figure
box = interactFigure(model, rows=2, height=650, **line_styles)

In [8]:
# Genetic result browser
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Profit', ascending=False)


def on_changed(event, grid):
    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k+1].to_dict('records')[0]
    param = dict(filter(lambda x: x[0] in G.args, selected.items()))

    updateSliders(box.children[0].children, **param)
    updateFigure(box.children[1], **model(**param)[1])


grid = show_grid(X, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False}, 
                 column_options={'defaultSortAsc': False})
grid.on('selection_changed', on_changed)

VBox([box, grid])

In [9]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'index',
                              'range': [0, 1279],
                              'values': array([1140,  349,  389, ..., 1166, 1183, 1277])},
                             {'label': 'Period',
                              'range': [0, 49835],
                              'values': array([27222, 26990, 31000, ...,     0, 37002,     0])},
                             {'label': 'StdDev',
                              'range': [0.0, 3.9992371171562824],
                              'values': array([2.39439805, 2.39983037, 2.48594046, ..., 2.36125938, 0.        ,
                                               2.35712531])},
                             {'label': 'Threshold',
                              'range': [0, 39],
                              'values': array([39, 39,  2, ...,  0, 28, 16])},
                             {'label': 'Profit',
                              'range': [-20440.722478356463, 7132.065173835399],
 